<a href="https://colab.research.google.com/github/bitdough/learning/blob/master/Link_counter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Twitter External Link Analysis Tool

This code takes a list of twitter usernames, iterates over them to find tweets where they shared links, and then sums up the base URLs of everyone's links combined and turns it into a bar graph. Please check code documentation for usage guidance. The code does take a bit to run depending on your tweet limit and how many accounts you pull. Note: this code writes some CSV's to the hosted drive file to store the data. Running this code repeatedly without refreshing each time before hand may add extra links to the end visualization. 

##Instructions to run

1.   Click Open in playground mode in top-left corner
2.   Write your list of usernames to analyze between the brackets below using the format described.
3.   Click Runtime in menu at top, click run all.
4.   Visualization should be at the bottom




In [0]:
#put accounts in between the brackets, comma seperated, without the @sign. ie ["jack", "realDonaldtrump", "Blacksocialists"]
#don't do a ton of accounts or it will take forever
sourceAccounts= ["PUT YOUR ACCOUNTS HERE" , "DIRECTIONS ABOVE"]

# New Section

In [0]:
%%capture
!pip install twint
#this prevents async problems/ runtime errors
#https://markhneedham.com/blog/2019/05/10/jupyter-runtimeerror-this-event-loop-is-already-running/
!pip install nest_asyncio

In [0]:
#This code takes a list of twitter usernames, iterates over them to find tweets where they shared links,
#and then sums up the base URLs of everyones links combined and turns it into a matplotlib graph.
#I put a bunch of code documentation in and it really will help you use this.
#the code does take a bit to run depending on your tweet limit and how many accounts you pull

import pandas as pd
import re

from urllib.parse import urlparse
from urllib.request import urlopen

import twint #you may need to install this first if you haven't!

import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt

import csv 
import os

import nest_asyncio
nest_asyncio.apply()



with open('all_urls.csv', 'wb') as f:
  pass

allURLs = []
for username in sourceAccounts:
    c = twint.Config()
    print("pulling tweets for " + str(username) + "...")
    print("ignore twint errors")
    c.Username = username
    c.Hide_output = True #makes the command line less noisy
    c.Limit = 500 #maximum number of tweets to pull per account
    c.Store_object = True
    #only selects tweets that have links
    c.Links = "include"


    baseURLs = []
    twint.run.Search(c)
    tweets = twint.output.tweets_list
    for tweet in tweets:
        #urls is a class in the twint tweet objects to see all classes: dir(tweet)
        for URL in tweet.urls:

            parsed_uri = urlparse(URL)
            baseURL = str('{uri.netloc}'.format(uri=parsed_uri)) #gets the base URL
            if baseURL[:7] == 'twitter': #ignores RTs as links
                pass
            elif baseURL[:4] == "www.": #strips www for a e s t h e t i c
                baseURLs.append([username, baseURL[4:]])
            else:
                baseURLs.append([username, baseURL])

    # all_urls += baseURLs
    # I added this in case it gets slow in pulling the list so you can stop at any point and then just
    # edit your sourceAccounts list to get rid of the one's you've already done.
    with open('all_urls.csv','a', newline='') as f:
        for baseURL in baseURLs:
            writer = csv.writer(f)
            writer.writerow(baseURL)



all_urls = pd.read_csv('all_urls.csv', names = ['username','URL'])
# all_urls = all_urls.DataFrame(names = ['username','URL'])

print("total tweets pulled: " + str(len(all_urls)))

In [0]:
labels = ['Base URL', 'Frequency']
countedURLs = all_urls['URL'].value_counts()
countedURLs.to_csv('countedURLs.csv')

top_urls = countedURLs.iloc[:10]
top_urls = top_urls[::-1] #makes it descending

y_pos = np.arange(len(top_urls))
performance = top_urls
print(performance)
baseURLs =  top_urls.index
print(baseURLs)
plt.barh(y_pos, performance, align='center', alpha=0.5)
plt.yticks(y_pos, baseURLs)
plt.xlabel('Frequency of Links')
plt.title('Most Frequent External Links of all Handles Tested')

plt.show()
